# GitHub Integration with MCP

This notebook demonstrates how to build a GitHub manager using the Model Context Protocol (MCP) with FastMCP.

## Overview
- Initialize an MCP server for GitHub operations
- Create tools to check repo status, manage PRs, and handle review requests
- Perfect for AI agents that need GitHub integration

## Prerequisites
- `fastmcp` and `PyGithub` installed
- `GITHUB_TOKEN` environment variable set

In [2]:
# Install required dependencies
import subprocess
import sys

packages = ["fastmcp", "PyGithub", "python-dotenv"]
for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✅ Dependencies installed")

✅ Dependencies installed


In [3]:
# Import Required Libraries
from mcp.server.fastmcp import FastMCP
from github import Github
import os
import datetime
import dotenv

dotenv.load_dotenv()

# Initialize Server
mcp = FastMCP("Pamudu-GitHub-Manager")

# Setup Authentication
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
if not GITHUB_TOKEN:
    print("⚠️ Warning: Using placeholder token for testing")
    GITHUB_TOKEN = "your_token_here"

g = Github(GITHUB_TOKEN)
USER = "pamudu1111"

print("✅ Server initialized and authenticated")

✅ Server initialized and authenticated


C:\Users\PK\AppData\Local\Temp\ipykernel_33060\3817335537.py:19: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(GITHUB_TOKEN)


## Tool 1: Repository Status

Check remote repository status similar to `git status` but for cloud.
Shows default branch, last commit, and PR/issue counts.

In [4]:
@mcp.tool()
def get_repo_status(repo_name: str) -> str:
    """
    Acts like 'git status' but for the remote GitHub repo.
    Shows: Default branch, Last commit, and Open PR count.
    """
    try:
        repo = g.get_user(USER).get_repo(repo_name)
        branch = repo.get_branch(repo.default_branch)
        commit = branch.commit
        
        status_report = f"""
📦 **REPO STATUS: {repo.name}**
--------------------------------
🔹 **Default Branch:** {repo.default_branch}
🔹 **Last Commit:** {commit.sha[:7]} - {commit.commit.message}
🔹 **Date:** {commit.commit.author.date}
🔹 **Open PRs:** {repo.get_pulls(state='open').totalCount}
🔹 **Open Issues:** {repo.get_issues(state='open').totalCount}
🔹 **Stars:** {repo.stargazers_count}
--------------------------------
"""
        return status_report
    except Exception as e:
        return f"❌ Error fetching status: {str(e)}"

## Tool 2: Pull Request Management

List your pull requests that are open, closed, or all.
Helps track PR activity across repositories.

In [5]:
@mcp.tool()
def list_my_prs(state: str = "open") -> str:
    """
    Lists Pull Requests created by me OR assigned to me.
    Args: state = 'open' or 'closed' or 'all'
    """
    query = f"author:{USER} type:pr state:{state}"
    prs = g.search_issues(query=query)
    
    report = [f"🚀 **My {state.upper()} Pull Requests:**"]
    
    if prs.totalCount == 0:
        return "No PRs found."

    for pr in prs[:5]:
        report.append(f"- [#{pr.number}] {pr.title} ({pr.repository.name})")
        report.append(f"  🔗 {pr.html_url}")
    
    return "\n".join(report)

## Tool 3: Review Requests

Check for pending PR reviews assigned to you.
Helps prioritize actionable items in your workflow.

In [6]:
@mcp.tool()
def check_review_requests() -> str:
    """
    Checks if anyone has requested my review on a PR.
    """
    query = f"review-requested:{USER} state:open type:pr"
    prs = g.search_issues(query=query)
    
    if prs.totalCount == 0:
        return "✅ You have no pending PR reviews."
        
    report = ["👀 **PRs Waiting for Your Review:**"]
    for pr in prs:
        report.append(f"- {pr.repository.name} #{pr.number}: {pr.title}")
        
    return "\n".join(report)

## Running the Server

Start the MCP server to make these tools available to AI agents.

In [7]:
# Run the server
if __name__ == "__main__":
    print("✅ GitHub Manager is running...")
    mcp.run()

✅ GitHub Manager is running...


RuntimeError: Already running asyncio in this thread